<a href="https://colab.research.google.com/github/SamuelAmrich/TLEIA-TLEOS/blob/master/Hydra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Nainstalovanie potrebnych dodatocnych kniznic

!pip3 install --upgrade pip

!pip3 install tensorflow
!pip3 install keras
!pip3 install opencv-python

!pip install split-folders
!pip install -q pyyaml h5py

     |████████████████████████████████| 1.5MB 2.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [ ]:
# Importovanie kniznic

# Zakladne kniznice pre pracu
import math, time, os, random, shutil

# Pre dodatocne spracovanie 
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL.Image
from PIL import Image
import cv2
from google.colab import drive

mpl.rcParams['figure.figsize'] = (12, 5)

# Pre neuronovu siet
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import layers
AUTOTUNE = tf.data.experimental.AUTOTUNE

import keras

Using TensorFlow backend.


In [ ]:
#Definicia pre vyplotenie jedneho obrazku

def plotImages(img):
    fig, axes = plt.subplots(1, 1, figsize=(5,5))
    axes.imshow(img[0])
    axes.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# Namapuje google drive

!mkdir raw_data 
drive.mount('raw_data/')
path = '/content/raw_data/My Drive/Space::Lab/ML_Left/'
print("Cesta k RAW datam je: " + path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at raw_data/
Cesta k RAW datam je: /content/raw_data/My Drive/Space::Lab/ML_Left/


In [ ]:
# A tereaz pekne odznovu

train_dir = "/content/raw_data/My Drive/Space::Lab/ML_Left/train"
validation_dir = "/content/raw_data/My Drive/Space::Lab/ML_Left/validation"


# Poscita pocet obrazkov

num_nothing_tr    =   len(os.listdir(train_dir+"/nothing"))
num_elves_tr      =   len(os.listdir(train_dir+"/elves"))
num_sprite_tr      =   len(os.listdir(train_dir+"/sprite"))
num_meteor_tr      =   len(os.listdir(train_dir+"/meteor"))

num_nothing_val   =   len(os.listdir(validation_dir+"/nothing"))
num_elves_val     =   len(os.listdir(validation_dir+"/elves"))
num_sprite_val     =   len(os.listdir(validation_dir+"/sprite"))
num_meteor_val     =   len(os.listdir(validation_dir+"/meteor"))

total_train       =   num_nothing_tr + num_elves_tr + num_sprite_tr +num_meteor_tr
total_val         =   num_nothing_val + num_elves_val + num_sprite_val + num_meteor_val

# Vypise ciselne pocty event-ov
print("--")
print('total training nothing images:', num_nothing_tr)
print('total training elves images:', num_elves_tr)
print('total training sprite images:', num_sprite_tr)
print('total training meteor images:', num_meteor_tr)
print("--")
print('total validation nothing images:', num_nothing_val)
print('total validation elves images:', num_elves_val)
print('total validation sprite images:', num_sprite_val)
print('total validation meteor images:', num_meteor_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)


#Nastavuje vlastnosti obrazkov a modelu

batch_size    =   55
epochs        =   10
IMG_HEIGHT    =   200
IMG_WIDTH     =   250
# Moje rozlisenie 250 x 200  /// Povodne rozlisenie 150 x 150

# Augmentacie obrazkov

image_gen_train = ImageDataGenerator(
                    rescale               =   1./255,       #preskalovanie
                    rotation_range        =   0,            #rotacia okolo stredu
                    width_shift_range     =   0.0,         #posunutie/roztiahnutie do sirky 
                    height_shift_range    =   0.0,         #posunutie/roztiahnutie do vysky
                    horizontal_flip       =   True,         #preklopenie obrazka okolo |
                    zoom_range            =   0.0,          #priblizenie obrazu
                    )

# Nacitanie trenovacich obrazkov do modelu
image_gen_train = ImageDataGenerator(rescale=1./255)
image_gen_val = ImageDataGenerator(rescale=1./255)

train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH))

# Vygenreuje validovacie obrazky pre neuronku

val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=validation_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH))

--
total training nothing images: 2650
total training elves images: 5
total training sprite images: 325
total training meteor images: 85
--
total validation nothing images: 400
total validation elves images: 5
total validation sprite images: 100
total validation meteor images: 20
--
Total training images: 3065
Total validation images: 525
Found 3065 images belonging to 4 classes.
Found 525 images belonging to 4 classes.


In [ ]:

#Podme to navazit 

weight_for_0 = (1 / num_elves_tr)*(total_train)/2.0 
weight_for_1 = (1 / num_meteor_tr)*(total_train)/2.0
weight_for_2 = (1 / num_nothing_tr)*(total_train)/2.0 
weight_for_3 = (1 / num_sprite_tr)*(total_train)/2.0

#Class 0 = Elves
#Class 1 = Meteor
#Class 2 = Nothing
#Class 3 = Sprite

print("weight_for_0 = "+str(weight_for_0))
print("weight_for_1 = "+str(weight_for_1))
print("weight_for_2 = "+str(weight_for_2))
print("weight_for_3 = "+str(weight_for_3))

class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2, 3: weight_for_3}


weight_for_0 = 306.5
weight_for_1 = 18.02941176470588
weight_for_2 = 0.5783018867924528
weight_for_3 = 4.7153846153846155


In [ ]:
# Načítanie Cerberus modelu

tleia_model_hydra = tf.keras.models.load_model(path+"/tleia_model_hydra.h5")

"""
#MODEL XCeption

tleia_model_hydra = tf.keras.applications.Xception(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    pooling=None,
    classes=4,
    classifier_activation="softmax",)
"""

'\n#MODEL XCeption\n\ntleia_model_hydra = tf.keras.applications.Xception(\n    include_top=True,\n    weights=None,\n    input_tensor=None,\n    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),\n    pooling=None,\n    classes=4,\n    classifier_activation="softmax",)\n'

In [ ]:
#Skompiluj model

METRICS = [
      keras.metrics.Accuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      #keras.metrics.SpecificityAtSensitivity(0.4, name="SpecATSen")
      ]
     
tleia_model_hydra.compile(optimizer='adam',   
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=METRICS) #["accuracy"]

tleia_model_hydra.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 200, 250, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 99, 124, 32)  864         input_9[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 99, 124, 32)  128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 99, 124, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
#Trenovanie neuronky

history = tleia_model_hydra.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    class_weight=class_weight)

Epoch 1/10
55/55 [==============================] - 227s 4s/step - loss: 1.4589 - accuracy: 0.3233 - precision: 0.9599 - recall: 0.9599 - auc: 0.9735 - val_loss: 0.6138 - val_accuracy: 0.3235 - val_precision: 0.9599 - val_recall: 0.9599 - val_auc: 0.9735
Epoch 2/10
55/55 [==============================] - 227s 4s/step - loss: 1.4601 - accuracy: 0.3236 - precision: 0.9599 - recall: 0.9599 - auc: 0.9735 - val_loss: 0.6133 - val_accuracy: 0.3237 - val_precision: 0.9599 - val_recall: 0.9599 - val_auc: 0.9735
Epoch 3/10
55/55 [==============================] - 226s 4s/step - loss: 1.4515 - accuracy: 0.3239 - precision: 0.9599 - recall: 0.9599 - auc: 0.9735 - val_loss: 0.6138 - val_accuracy: 0.3241 - val_precision: 0.9599 - val_recall: 0.9599 - val_auc: 0.9735
Epoch 4/10
55/55 [==============================] - 228s 4s/step - loss: 1.4579 - accuracy: 0.3242 - precision: 0.9599 - recall: 0.9599 - auc: 0.9735 - val_loss: 0.6148 - val_accuracy: 0.3244 - val_precision: 0.9599 - val_recall: 0.959

In [ ]:
tleia_model_hydra.save(
    path+"/tleia_model_hydra/", overwrite=True, include_optimizer=True, save_format=True,
    signatures=None, options=None)


tleia_model_hydra.save(path+"/tleia_model_hydra.h5")

INFO:tensorflow:Assets written to: /content/raw_data/My Drive/Space::Lab/ML_Left//tleia_model_hydra/assets


--------------------------------------
Výsledky v 10 epoch pre rôzne modely::

XCeption: 178s 3s/step - loss: 0.6071 - accuracy: 0.9641 - val_loss: 0.6578 - val_accuracy: 0.7616

NasNetMobile: 173s 3s/step - loss: 0.6075 - accuracy: 0.9631 - val_loss: 0.7509 - val_accuracy: 0.4141

NasNetLarge: 211s 4s/step - loss: 0.6077 - accuracy: 0.9618 - val_loss: 0.8007 - val_accuracy: 0.1899

VGG19: 177s 3s/step - loss: 0.6320 - accuracy: 0.8648 - val_loss: 0.6583 - val_accuracy: 0.7596

--------------------------------------
Výsledky v maxime pre rôzne modely::

XCeption:

NasNetMobile:

NasNetLarge:

VGG19: